In [1]:
import tensorflow as tf

2024-08-18 12:26:22.676710: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-18 12:26:22.676828: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-18 12:26:22.810162: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import pickle
with open('/kaggle/input/hand-gesture-dataset/labels.pkl','rb') as f:
    y=pickle.load(f)

In [3]:
with open('/kaggle/input/hand-gesture-dataset/tensors.pkl', 'rb') as f:
    x=pickle.load(f)

In [4]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.2)

In [5]:
#had to create the model again because for some reason (maybe due to naming the file as .h5) the model could not be loaded properly

vgg16=tf.keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
vgg16.trainable=False
input_layer=tf.keras.layers.Input(shape=(224, 224, 3))
x=vgg16(input_layer)
x=tf.keras.layers.Flatten()(x)
no=256*10
# x=tf.keras.layers.Dense(units=self.config.params_classes, activation= 'relu')(x)
while (no>10):
    x=tf.keras.layers.Dense(units=no, activation='relu')(x)
    x=tf.keras.layers.Dropout(0.2)(x)
    no=int(no/4)
    
x=tf.keras.layers.Dense(units=10, activation='softmax')(x)
model=tf.keras.Model(inputs=input_layer, outputs=x)
model.summary()

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2560)           │    64,227,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2560)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 640)            │     1,639,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 640)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 160)            │       102,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 160)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 40)             │         6,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 40)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │           410 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 80,690,978 (307.81 MB)

 Trainable params: 65,976,290 (251.68 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [6]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [8]:
from tensorflow.keras.callbacks import EarlyStopping
model.fit(x_train, y_train, epochs=200, callbacks=[EarlyStopping(restore_best_weights=True, monitor='loss', patience=10)])

Epoch 1/200
  1/132 ━━━━━━━━━━━━━━━━━━━━ 1:20:12 37s/step - loss: 16.5202

I0000 00:00:1723984081.578568     111 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1723984081.601609     111 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


132/132 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step - loss: 9.9168

W0000 00:00:1723984108.318778     110 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


132/132 ━━━━━━━━━━━━━━━━━━━━ 64s 209ms/step - loss: 9.8781
Epoch 2/200
132/132 ━━━━━━━━━━━━━━━━━━━━ 17s 131ms/step - loss: 0.9174
Epoch 3/200
132/132 ━━━━━━━━━━━━━━━━━━━━ 18s 134ms/step - loss: 0.4185
Epoch 4/200
132/132 ━━━━━━━━━━━━━━━━━━━━ 18s 134ms/step - loss: 0.3590
Epoch 5/200
132/132 ━━━━━━━━━━━━━━━━━━━━ 17s 132ms/step - loss: 0.2009
Epoch 6/200
132/132 ━━━━━━━━━━━━━━━━━━━━ 17s 131ms/step - loss: 0.1186
Epoch 7/200
132/132 ━━━━━━━━━━━━━━━━━━━━ 17s 129ms/step - loss: 0.1224
Epoch 8/200
132/132 ━━━━━━━━━━━━━━━━━━━━ 17s 129ms/step - loss: 0.2128
Epoch 9/200
132/132 ━━━━━━━━━━━━━━━━━━━━ 18s 133ms/step - loss: 0.0701
Epoch 10/200
132/132 ━━━━━━━━━━━━━━━━━━━━ 17s 130ms/step - loss: 0.1048
Epoch 11/200
132/132 ━━━━━━━━━━━━━━━━━━━━ 17s 129ms/step - loss: 0.1813
Epoch 12/200
132/132 ━━━━━━━━━━━━━━━━━━━━ 17s 129ms/step - loss: 0.2320
Epoch 13/200
132/132 ━━━━━━━━━━━━━━━━━━━━ 17s 129ms/step - loss: 0.2296
Epoch 14/200
132/132 ━━━━━━━━━━━━━━━━━━━━ 17s 129ms/step - loss: 0.3278
Epoch 15/200


In [9]:
y_pred=model.predict(x_test)

 2/33 ━━━━━━━━━━━━━━━━━━━━ 4s 156ms/step 

W0000 00:00:1723984713.426536     111 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


33/33 ━━━━━━━━━━━━━━━━━━━━ 29s 896ms/step


W0000 00:00:1723984742.086133     109 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


In [11]:
y_pred_actual=[]
y_test_actual=[]
for i in range(len(y_pred)):
    y_pred_actual.append(y_pred[i].argmax())
    y_test_actual.append(y_test[i].argmax())

In [12]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_pred_actual, y_test_actual)
print(accuracy)

0.9933269780743565


In [14]:
model.save('/kaggle/working/trained_model.keras')